In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as plt_img



In [3]:
import os
from utils import mytransforms as my_transforms
import torch
from datasets import get_dataset
from utils import coco_eval, coco_utils, utils2
from tqdm import tqdm
from evaluate_LSC import *
import torchvision.transforms as transforms

In [4]:
def val(dataset_it, coco_parts_dataset, epoch, device, only_eval=False, lsc_eval=False):
    # coco_parts_evaluator.coco_eval["bbox"].params.catIds = [1]
    # loop all datasets in gtLabelFile and compare to respective dataset in inLabelFile
    if lsc_eval:
        results_lsc = {
          'header': ('number', 'filename', 'SymBestDice', 'FgBgDice', 'aDiC', 'DiC',
                     'Pa', 'Pa±1')
        }
        stats_lsc = {'header': ('measure', 'mean', 'std')}

        SymBestDice = []
        FgBgDice = []
        absDiC = []
        DiC = []
        Pa = []
        PaPM1 = []
        filenames = []
        
    for sample in tqdm(dataset_it):

        
        output = plt.imread(f"./Komatsuna_DIP1/{sample['image_name'][0]}")

        if True:
            batch_size = 1


        for j in range(batch_size):
            part_img = output
            try:
                if True:
                    # now call the scoring routines and append results to respective lists
                    inLabel = part_img
                    gtLabel = transforms.functional.to_pil_image(sample['parts_instances'][0])
                    gtLabel = np.asarray(gtLabel)
                    SymBestDice.append(
                      np.minimum(BestDice(inLabel, gtLabel), BestDice(gtLabel, inLabel)))
                    FgBgDice.append(FGBGDice(inLabel, gtLabel))
                    absDiC.append(absDiffCount(inLabel, gtLabel))
                    DiC.append(DiffCount(inLabel, gtLabel))
                    Pa.append(PercentageAgree(inLabel, gtLabel))
                    PaPM1.append(PercentageAgreePM1(inLabel, gtLabel))
            except:
                print(inLabel.shape)
                print(gtLabel.shape)
        
    if True:
            # store results in dictionary
        zipped = zip(
            range(1, dataset_it.__len__() + 1), filenames, SymBestDice, FgBgDice, absDiC, DiC, Pa, PaPM1)
        groupresults = {"LSC": list(zipped)}
        groupstats = {
            "LSC": [('SymBestDice', np.mean(SymBestDice), np.std(SymBestDice)),
                  ('FgBgDice', np.mean(FgBgDice), np.std(FgBgDice)),
                  ('aDiC', np.mean(absDiC), np.std(absDiC)),
                  ('DiC', np.mean(DiC), np.std(DiC)),
                  ('Pa', np.mean(Pa), np.std(Pa)),
                  ('Pa±1', np.mean(PaPM1), np.std(PaPM1))]
        }
        results_lsc.update(groupresults)
        stats_lsc.update(groupstats)
    coco_parts_evaluator = None
    return coco_parts_evaluator, results_lsc, stats_lsc


In [5]:
PATH = os.path.join(os.path.dirname(os.getcwd()),"LeafSAM", "dataset")
dataset_opts = dict(
    root_dir=os.path.join(PATH, "Komatsuna"),
    split="train",
    transform=my_transforms.get_transform([
                {
                    'name': 'ToTensor',
                    'opts': {
                        'keys': ['image', 'global_instances', 'global_labels', 'parts_instances', 'parts_labels'],
                        'type': [torch.FloatTensor, torch.ByteTensor, torch.ByteTensor, torch.ByteTensor, torch.ByteTensor],
                    }
                },
            ])
)
komatsuna_train_dataset = get_dataset("komatsuna" ,dataset_opts)

komatsuna_train_dataset_it = torch.utils.data.DataLoader(
    komatsuna_train_dataset, batch_size=1, shuffle=True, drop_last=True, num_workers=1, pin_memory=False)

coco_parts_dataset = coco_utils.convert_parts_to_coco_api(komatsuna_train_dataset_it.dataset)

No# of image files: 300
No# of segment files: 300
Komatsuna Dataset created [train]
creating index...
index created!


In [6]:
komatsuna_train_dataset[1]["image_name"]

'rgb_00_000_01.png'

In [9]:
import time

# seconds passed since epoch
seconds =  time.time()

# convert the time in seconds since the epoch to a readable format
start_time = time.ctime(seconds)

coco_eval_results, lsc_r, lsc_s = val(komatsuna_train_dataset_it, coco_parts_dataset, epoch=-1, device=None, only_eval=True, lsc_eval=True)

# seconds passed since epoch
seconds =  time.time()

# convert the time in seconds since the epoch to a readable format
end_time = time.ctime(seconds)

print("Start time:", start_time)
print("End time:", end_time)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:07<00:00, 37.54it/s]

Start time: Wed Aug  7 21:10:28 2024
End time: Wed Aug  7 21:10:36 2024


In [10]:
lsc_s

{'header': ('measure', 'mean', 'std'),
 'LSC': [('SymBestDice', 0.6469518971807466, 0.18441370749224897),
  ('FgBgDice', 0.8629745496917404, 0.16047764248233926),
  ('aDiC', 1.1766666666666667, 0.9620060059872576),
  ('DiC', -0.91, 1.2173331507849443),
  ('Pa', 0.26666666666666666, 0.44221663871405337),
  ('Pa±1', 0.66, 0.4737087712930804)]}